<a href="https://colab.research.google.com/github/jrebull/ChileSistemasRecomendacion/blob/main/3_practico_implicit_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico 3 - Recomendación basada en feedback implícito.

Magister en Inteligencia Artificial - UC

**Profesor:** Vicente Domínguez

**Nombre Alumno:**

In [ ]:
!gdown 1gmOrtPpZpHJ0HeBwtne-kA8Bll4rFWW7
!gdown 1bnLJUEIRx13k4nxN7x7Fa-3L37rXre73
!gdown 1i92TtKsgf_3ffef8EVLH9NNArxvF-cMo

Downloading...
From: https://drive.google.com/uc?id=1gmOrtPpZpHJ0HeBwtne-kA8Bll4rFWW7
To: /content/u.item
100% 236k/236k [00:00<00:00, 69.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bnLJUEIRx13k4nxN7x7Fa-3L37rXre73
To: /content/u2.base
100% 1.58M/1.58M [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i92TtKsgf_3ffef8EVLH9NNArxvF-cMo
To: /content/u2.test
100% 395k/395k [00:00<00:00, 93.3MB/s]


In [ ]:
!pip3 install implicit --upgrade

In [ ]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sparse

In [ ]:
columns = ['movieid', 'title', 'release_date', 'video_release_date', \
           'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

In [ ]:
# Primero creamos el dataframe con los datos
df_train = pd.read_csv('u2.base',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)

# rating >= 3 , relevante (1) y rating menor a 3 es no relevante (0)
df_train.rating = [1 if x >=3 else 0 for x in df_train.rating ]

In [ ]:
df_train

,userid,itemid,rating,timestamp
0,1,3,1,878542960
1,1,4,1,876893119
2,1,5,1,889751712
3,1,6,1,887431973
4,1,7,1,875071561
...,...,...,...,...
79995,943,1067,0,875501756
79996,943,1074,1,888640250
79997,943,1188,1,888640250
79998,943,1228,1,888640275


In [ ]:
# Cargamos el dataset con los items
df_items = pd.read_csv('u.item',
                        sep='|',
                        index_col=0,
                        names = columns,
                        header=None,
                        encoding='latin-1')

df_items

,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieid,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
# Cargamos el dataset de testing
df_test = pd.read_csv('u2.test',
                      sep='\t',
                      names=['userid', 'itemid', 'rating', 'timestamp'],
                      header=None)


# rating >= 3 es relevante (1) y rating menor a 3 es no relevante (0)
df_test.rating = [1 if x >=3 else 0 for x in df_test.rating ]


user_items_test = {}

for row in df_test.itertuples():
  if row[1] not in user_items_test:
    user_items_test[row[1]] = []

    user_items_test[row[1]].append(row[2])

In [ ]:
user_items_test

{1: [1],
 2: [1],
 3: [181],
 4: [210],
 5: [21],
 6: [12],
 7: [23],
 8: [82],
 9: [276],
 10: [11],
 11: [24],
 12: [133],
 13: [1],
 14: [12],
 15: [13],
 16: [1],
 17: [117],
 18: [4],
 19: [210],
 20: [94],
 21: [5],
 22: [21],
 23: [7],
 24: [8],
 25: [116],
 26: [1],
 27: [9],
 28: [11],
 29: [79],
 30: [2],
 31: [32],
 32: [9],
 33: [245],
 34: [292],
 35: [258],
 36: [288],
 37: [27],
 38: [69],
 39: [269],
 40: [245],
 41: [175],
 42: [1],
 43: [3],
 44: [1],
 45: [13],
 46: [50],
 47: [258],
 48: [98],
 49: [13],
 50: [268],
 51: [83],
 52: [13],
 53: [7],
 54: [1],
 55: [117],
 56: [11],
 57: [8],
 58: [11],
 59: [1],
 60: [8],
 61: [258],
 62: [1],
 63: [15],
 64: [11],
 65: [15],
 66: [24],
 67: [24],
 68: [25],
 69: [42],
 70: [8],
 71: [14],
 72: [2],
 73: [32],
 74: [7],
 75: [1],
 76: [7],
 77: [15],
 78: [257],
 79: [19],
 80: [100],
 81: [1],
 82: [3],
 83: [2],
 84: [1],
 85: [10],
 86: [270],
 87: [21],
 88: [300],
 89: [7],
 90: [18],
 91: [31],
 92: [4],
 93: [1

In [ ]:
# Definicion de métricas (No editar)
# Obtenido de https://gist.github.com/bwhite/3726239
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [ ]:
user_items = {}
itemset = set()

for row in df_train.itertuples():
    if row[1] not in user_items:
        user_items[row[1]] = []

    user_items[row[1]].append(row[2])
    itemset.add(row[2])

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

matrix = sparse.csr_matrix(sparse_matrix.T)

user_ids = {key: i for i, key in enumerate(user_items.keys())}
user_item_matrix = matrix.T.tocsr()

In [ ]:
user_item_matrix.toarray()

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
def evaluate_model(model, n):
  mean_map = 0.
  mean_ndcg = 0.
  for u in user_items_test.keys():
    rec = model.recommend(u, user_item_matrix[u], n)[0]
    rel_vector = [np.isin(user_items_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, n)

  mean_map /= len(user_items_test)
  mean_ndcg /= len(user_items_test)

  return mean_map, mean_ndcg

In [ ]:
def show_recommendations(model, user, n):
  recommendations = model.recommend(userid=user, user_items=user_item_matrix[user], N=n)[0]
  return df_items.loc[recommendations]['title']


In [ ]:
def show_similar_movies(model, item, n=10):
  sim_items = model.similar_items(item, n)[0]
  return df_items.loc[sim_items]['title']


# ALS

In [ ]:
model_als = implicit.als.AlternatingLeastSquares(factors=10, iterations=10, use_gpu=False)
model_als.fit(user_item_matrix)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
maprec, ndcg = evaluate_model(model_als, n=10)
print('map: {}\nndcg@10: {}'.format(maprec, ndcg))

map: 0.015313935681470138
ndcg@10: 0.015313935681470138


# BPR

In [ ]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=10, use_gpu=False)
model_bpr.fit(user_item_matrix)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
maprec, ndcg = evaluate_model(model_bpr, n=10)
print('map: {}\nndcg@10: {}'.format(maprec, ndcg))

map: 0.010719754977029096
ndcg@10: 0.010719754977029096


Comente (2 puntos)
- Cuál de los dos métodos entrega mejores resultados en terminos de `ndcg@10` y `map`?

##### RESPONDER AQUI ##############################

# Analisis de sensilidad ALS (5 puntos)
- Modificar la cantidad de factores latentes (10, 50, 100, 150, 200, 300 y 500) reportando valores de ndcg@10 y MAP.

In [ ]:
#### ESCRIBIR CODIGO AQUI #############################


Comente (2 ptos)
- ¿Cuál es la cantidad óptima de factores latentes utilizando ALS?

##### RESPONDER AQUI ##############################


# Analisis de senbilidad BPR (5 puntos)
- Modificar la cantidad de factores latentes (10, 50, 100, 150 , 200 y 500) reportando valores de ndcg@10 y MAP.

In [ ]:
#### ESCRIBIR CODIGO AQUI #############################

Comente (2 ptos)
- ¿Cuál es la cantidad óptima de factores latentes utilizando BPR?

## Análisis general (2 ptos)
- ¿Cuál de los dos métodos considerando la cantidad de factores latentes optimos tiene mejores resultados en términos de ranking (ndcg@10 y MAP)?

RESPONDER AQUI

## Análisis de contenido (2 ptos)
Buscar 10 películas similares a `Star Wars (1977)` utilizando el modelo BPR con la cantidad optima de factores latentes obtenido de la actividad anterior.


In [ ]:
#### ESCRIBIR CODIGO AQUI #############################

¿Por qué los resultados no hacen mucho sentido? Explique la intuición.

RESPONDER AQUI
